In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import random as python_random
import json
import argparse
import time
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Embedding, LSTM, Bidirectional
from keras.initializers import Constant
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import SGD, Adam, Adagrad, Adamax, RMSprop
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf
# Make reproducible as much as possible
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
def read_corpus(corpus_file):
    '''Read in review data set and returns docs and labels'''
    documents = []
    labels = []
    count = 0
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            tokens = line.strip()
            documents.append(" ".join(tokens.split()[3:]).strip())
            # 6-class problem: books, camera, dvd, health, music, software
            labels.append(tokens.split()[0])
    for label in labels:
      if label == 'software'
    return documents, labels

def read_embeddings(embeddings_file):
    '''Read in word embeddings from file and save as numpy array'''
    embeddings = json.load(open(embeddings_file, 'r'))
    return {word: np.array(embeddings[word]) for word in embeddings}

def get_emb_matrix(voc, emb):
    '''Get embedding matrix given vocab and the embeddings'''
    num_tokens = len(voc) + 2
    word_index = dict(zip(voc, range(len(voc))))
    # Bit hacky, get embedding dimension from the word "the"
    embedding_dim = len(emb["the"])
    # Prepare embedding matrix to the correct size
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = emb.get(word)
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    # Final matrix with pretrained embeddings that we can feed to embedding layer
    return embedding_matrix

In [ ]:
def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    print("Embedding dimension: ", embedding_dim)
    num_tokens = len(emb_matrix)
    print("Num tokens: ", num_tokens)
    num_labels = len(Y_train[0])
    print("Num labels: ", num_labels)
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix), trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    # raise NotImplementedError("Add LSTM layer(s) here")
    model.add(LSTM(75))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

In [ ]:
def train_model(model, X_train, Y_train, X_dev, Y_dev):
    '''Train the model here. Note the different settings you can experiment with!'''
    # Potentially change these to cmd line args again
    # And yes, don't be afraid to experiment!
    verbose = 1
    batch_size = 16
    epochs = 50
    # Early stopping: stop training when there are three consecutive epochs without improving
    # It's also possible to monitor the training loss with monitor="loss"
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    # Finally fit the model to our data
    model.fit(X_train, Y_train, verbose=verbose, epochs=epochs, callbacks=[callback], batch_size=batch_size, validation_data=(X_dev, Y_dev))
    # Print final accuracy for the model (clearer overview)
    test_set_predict(model, X_dev, Y_dev, "dev")
    return model

In [ ]:
def test_set_predict(model, X_test, Y_test, ident):
    '''Do predictions and measure accuracy on our own test set (that we split off train)'''
    # Get predictions using the trained model
    Y_pred = model.predict(X_test)
    # Finally, convert to numerical labels to get scores with sklearn
    Y_pred = np.argmax(Y_pred, axis=1)
    # If you have gold data, you can calculate accuracy
    Y_test = np.argmax(Y_test, axis=1)
    print('Accuracy on own {1} set: {0}'.format(round(accuracy_score(Y_test, Y_pred), 3), ident))

In [ ]:
def main():
    '''Main function to train and test neural network'''
    # Read in the data and embeddings
    X_train, Y_train = read_corpus('/content/gdrive/MyDrive/AS5/train.txt')
    X_dev, Y_dev = read_corpus('/content/gdrive/MyDrive/AS5/dev.txt')
    embeddings = read_embeddings('/content/gdrive/MyDrive/AS5/glove_reviews.json')

    # Transform words to indices using a vectorizer
    vectorizer = TextVectorization(standardize=None, output_sequence_length=50)
    # Use train and dev to create vocab - could also do just train
    text_ds = tf.data.Dataset.from_tensor_slices(X_train + X_dev)
    vectorizer.adapt(text_ds)
    # Dictionary mapping words to idx
    voc = vectorizer.get_vocabulary()
    emb_matrix = get_emb_matrix(voc, embeddings)

    # Transform string labels to one-hot encodings
    encoder = LabelBinarizer()
    Y_train_bin = encoder.fit_transform(Y_train)  # Use encoder.classes_ to find mapping back
    Y_dev_bin = encoder.fit_transform(Y_dev)

    # Create model
    model = create_model(Y_train, emb_matrix)

    # Transform input to vectorized input
    X_train_vect = vectorizer(np.array([[s] for s in X_train])).numpy()
    X_dev_vect = vectorizer(np.array([[s] for s in X_dev])).numpy()

    # Train the model
    model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin)

    test_file = False
    # Do predictions on specified test set
    if test_file:
        # Read in test set and vectorize
        X_test, Y_test = read_corpus('test.txt')
        Y_test_bin = encoder.fit_transform(Y_test)
        X_test_vect = vectorizer(np.array([[s] for s in X_test])).numpy()
        # Finally do the predictions
        test_set_predict(model, X_test_vect, Y_test_bin, "test")

In [ ]:
t0 = time.time()
main()
train_time = time.time() - t0
print("Training time: ", train_time)

Embedding dimension:  300
Num tokens:  38113
Num labels:  6
Epoch 1/50
313/313 [==============================] - 8s 18ms/step - loss: 1.7466 - accuracy: 0.2656 - val_loss: 1.6796 - val_accuracy: 0.3660
Epoch 2/50
313/313 [==============================] - 5s 15ms/step - loss: 1.4849 - accuracy: 0.4474 - val_loss: 1.2816 - val_accuracy: 0.5180
Epoch 3/50
313/313 [==============================] - 5s 16ms/step - loss: 1.0954 - accuracy: 0.5730 - val_loss: 0.8739 - val_accuracy: 0.6740
Epoch 4/50
313/313 [==============================] - 5s 15ms/step - loss: 0.9058 - accuracy: 0.6532 - val_loss: 0.7202 - val_accuracy: 0.7420
Epoch 5/50
313/313 [==============================] - 5s 16ms/step - loss: 0.7684 - accuracy: 0.7292 - val_loss: 0.9130 - val_accuracy: 0.6340
Epoch 6/50
313/313 [==============================] - 5s 15ms/step - loss: 0.6731 - accuracy: 0.7666 - val_loss: 0.6306 - val_accuracy: 0.7740
Epoch 7/50
313/313 [==============================] - 5s 15ms/step - loss: 0.5737 

In [ ]:
# !python /content/gdrive/MyDrive/AS5/lfd_assignment5.py --train_file /content/gdrive/MyDrive/AS5/train.txt --dev_file /content/gdrive/MyDrive/AS5/dev.txt --embeddings /content/gdrive/MyDrive/AS5/glove_reviews.json